In [1]:
import json
import numpy as np
import time
import random
import pickle
from tqdm import tqdm
from AIs import manh, numpy_rl_reload
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import game
import rl

In [2]:
epoch = 10000  # Total number of epochs that will be done
max_memory = 1000  # Maximum number of experiences we are storing
number_of_batches = 8  # Number of batches per epoch
batch_size = 32  # Number of experiences we use for training per batch
width = 21  # Size of the playing field
height = 15  # Size of the playing field
cheeses = 40  # Number of cheeses in the game
opponent = manh  # AI used for the opponent

In [3]:
load = False
save = True
env = game.PyRat(width=width, height=height, opponent=opponent, cheeses=cheeses)
exp_replay = rl.ExperienceReplay(max_memory=max_memory)
model = rl.NLinearModels(env.observe()[0])

In [4]:
if load:
    model.load()

def play(model, epochs, train=True):

    win_cnt = 0
    lose_cnt = 0
    draw_cnt = 0
    win_hist = []
    cheeses = []
    steps = 0.
    last_W = 0
    last_D = 0
    last_L = 0
    
    # Define a loss function and optimizer
    ### CODE TO BE COMPLETED
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    for e in tqdm(range(epochs)):
        env.reset()
        game_over = False

        # Get the current state of the environment
        state = env.observe()

        # Play a full game until game is over
        while not game_over:
            # Do not forget to transform the input of model into torch tensor
            state = torch.FloatTensor(state)

            # Predict the Q value for the current state
            q_values = model(state)

            # Pick the next action that maximizes the Q value
            action = torch.argmax(q_values)
            
            # Apply action, get rewards and new state
            previous_state = state.detach().clone()
            state, reward, game_over= env.act(action)
            
            #print(f"Rewards: {reward} Actions: {action} ")
            
            # Statistics
            if game_over:
                steps += env.round
                if env.score > env.enemy_score:
                    win_cnt += 1
                elif env.score == env.enemy_score:
                    draw_cnt += 1
                else:
                    lose_cnt += 1
                cheese = env.score

            # Create an experience array using previous state, the performed action, the obtained reward and the new state. The vector has to be in this order.
            # Store in the experience replay buffer an experience and end game.
            # Do not forget to transform the previous state and the new state into torch tensor.
            # Create an experience array
            experience = [torch.FloatTensor(previous_state), action, reward, torch.FloatTensor(state)]

            # Store the experience in the experience replay buffer
            exp_replay.remember(experience,game_over)
            
        win_hist.append(win_cnt)  # Statistics
        cheeses.append(cheese)  # Statistics

        if train:

            # Train using experience replay. For each batch, get a set of experiences (state, action, new state) that were stored in the buffer. 
            # Use this batch to train the model.
            ### CODE TO BE COMPLETED
            
            running_loss = 0
            for b in range(number_of_batches):
                # get the inputs
                states, Q = exp_replay.get_batch(model, batch_size = batch_size)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + loss + backward + optimize
                # Forward pass
                output = model(states)
                # Compute loss
                loss = criterion(output, Q)
                # Backward pass
                loss.backward()
                # Update parameters
                optimizer.step()
                # statistics
                running_loss += loss.item()
            print('[%d] loss: %.3f' % (e + 1, running_loss / number_of_batches))
            running_loss = 0.0

        if (e+1) % 100 == 0:  # Statistics every 100 epochs
            cheese_np = np.array(cheeses)
            string = "Epoch {:03d}/{:03d} | Cheese count {} | Last 100 Cheese {}| W/D/L {}/{}/{} | 100 W/D/L {}/{}/{} | 100 Steps {}".format(
                        e,epochs, cheese_np.sum(), 
                        cheese_np[-100:].sum(), win_cnt, draw_cnt, lose_cnt, 
                        win_cnt-last_W, draw_cnt-last_D, lose_cnt-last_L, steps/100)
            print(string)
        
            steps = 0.
            last_W = win_cnt
            last_D = draw_cnt
            last_L = lose_cnt                

print("Training")
play(model,epoch, True)
if save:
    model.save()
print("Training done")

Training


  0%|                                                                                | 2/10000 [00:00<09:58, 16.71it/s]

[1] loss: 0.000
[2] loss: 0.003
[3] loss: 0.005


  0%|                                                                                | 4/10000 [00:00<10:28, 15.89it/s]

[4] loss: 0.007


...

[9993] loss: 0.031
[9994] loss: 0.019
[9995] loss: 0.038
[9996] loss: 0.014


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [11:36<00:00, 16.14it/s]

[9997] loss: 0.032
[9998] loss: 0.010
[9999] loss: 0.024
[10000] loss: 0.019
Epoch 9999/10000 | Cheese count 146718.0 | Last 100 Cheese 1865.5| W/D/L 3139/810/6051 | 100 W/D/L 49/9/42 | 100 Steps 69.33


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [11:36<00:00, 14.35it/s]

Training done


In [6]:
print("Testing")
play(model, epoch, False)
print("Testing done")

Testing


  1%|▉                                                                             | 117/10000 [00:01<01:53, 87.05it/s]

Epoch 099/10000 | Cheese count 1965.0 | Last 100 Cheese 1965.0| W/D/L 58/14/28 | 100 W/D/L 58/14/28 | 100 Steps 70.76


  2%|█▋                                                                            | 213/10000 [00:02<02:11, 74.64it/s]

Epoch 199/10000 | Cheese count 3916.0 | Last 100 Cheese 1951.0| W/D/L 111/23/66 | 100 W/D/L 53/9/38 | 100 Steps 70.68


...

Epoch 9799/10000 | Cheese count 188260.5 | Last 100 Cheese 1911.0| W/D/L 5123/1080/3597 | 100 W/D/L 51/11/38 | 100 Steps 70.33


 99%|████████████████████████████████████████████████████████████████████████████▎| 9914/10000 [02:04<00:01, 74.80it/s]

Epoch 9899/10000 | Cheese count 190132.0 | Last 100 Cheese 1871.5| W/D/L 5170/1089/3641 | 100 W/D/L 47/9/44 | 100 Steps 67.82


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:05<00:00, 79.54it/s]


Epoch 9999/10000 | Cheese count 192033.0 | Last 100 Cheese 1901.0| W/D/L 5222/1100/3678 | 100 W/D/L 52/11/37 | 100 Steps 72.42
Testing done


1st run -> W/D/L 0/0/1000 | 100 W/D/L 0/0/100
2nd run -> W/D/L 0/0/1000 | 100 W/D/L 0/0/100 
3rd run -> W/D/L 0/0/1000 | 100 W/D/L 0/0/100
4th run -> W/D/L 263/88/649 | 100 W/D/L 20/15/65
5th run -> W/D/L 421/119/460 | 100 W/D/L 44/9/47
6th run -> W/D/L 493/87/420 | 100 W/D/L 48/6/46
7th run -> W/D/L 516/99/385 | 100 W/D/L 56/6/38 
8th run -> W/D/L 513/115/372 | 100 W/D/L 43/12/45
9th run -> W/D/L 485/121/394 | 100 W/D/L 57/13/30
10th run -> W/D/L 506/111/383 | 100 W/D/L 47/9/44

5000 epoch ->W/D/L 1996/557/2447 | 100 W/D/L 38/12/50

W/D/L 376/89/535
462/123/415
464/110/426
